# **Basic**

In [42]:
import numpy as np
import contextlib
import weakref

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    __array_priority__ = 200    # 연산자 우선순위 설정 (Ex. np.array의 __add__보다 Variable의 __add__함수가 먼저 호출됨)
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다. step 09를 참조하세요.'.format(type(data)))
        
        self.data = data
        self.name = name
        self.grad = None    # gradient = 기울기
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수

    def __len__(self):
        return len(self.data)

    def __repr__(self): # Variable 클래스의 인스턴스를 print하는 방식 지정
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p})'
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = []
        seen_set = set()

        def add_func(f):    # 함수의 중복추가 방지. Ex : 141p 그림 16-4 의 0세대 square함수는 1세대의 두 square함수에 의해 두 번 추가된다. 이것을 방지
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()    # 함수를 가져온다
            gys = [output().grad for output in f.outputs]   # output.grad는 약하게 참조된 데이터에 접근할 수 없다.
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx    # 덮어쓰기 연산 x.grad += gx를 사용하면 문제가 생긴다.

                if x.creator is not None:   # 역전파가 끝나지 않았다면, 해당 함수를 추가한다.
                    add_func(x.creator)

            if not retain_grad: # 말단 변수(x0, x1 등) 이외에는 미분값을 유지하지 않는다.
                for y in f.outputs:
                    y().grad = None # y는 약한 참조(weakref), 이 코드가 실행되면 참조값 카운트가 0이되어 미분값 데이터가 메모리에서 삭제
    
    def cleargrad(self):
        self.grad = None

    @property   # x.shape()대신 x.shape로 마치 인스턴스 변수인 것처럼 메서드를 사용 가능.
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)    # with 블록안에서, name으로 지정한 Config 클래스의 속성이 value값으로 설정됨
    try:
        yield
    finally:
        setattr(Config, name, old_value)    # with 블록 바깥에서는, 원래 값인 old_value로 돌아감
        
class Function:
    def __call__(self, *inputs): # 가변 길이 입출력
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)  # list(xs) unpacking
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):    # 순전파
        raise NotImplementedError()
    
    def backward(self, gys):    
        raise NotImplementedError()

# **Func, Class**

In [44]:
def no_grad():
    return using_config('enable_backprop', False)

def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
def square(x):
    f = Square()
    return f(x)
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)

def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0,x1)

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy*x1, gy*x0
    
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
def neg(x):
    return Neg()(x)

class Sub(Function):
    def forward(self, x0, x1):
        y = x0 -x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy/x1
        gx1 = gy*(-x0 / x1 ** 2)
        return gx0, gx1
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

class Pow(Function):
    def __init__(self, c):
        self.c = c
        
    def forward(self, x):
        y = x**self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c-1) * gy
        return gx
    
def pow(x, c):
    return Pow(c)(x)

'''
Variable.__method__ = method는

class Variable:
    ...
    
    def __method__(self, other):
        return method(self, other)
        
와 같다.
'''
Variable.__mul__ = mul    # Variable * float
Variable.__add__ = add
Variable.__rmul__ = mul    # float * Variable
Variable.__radd__ = add
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

# step 11

In [ ]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


# Step 12

In [ ]:
# 첫 번째 개선
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)
print(f.inputs[0].data, f.inputs[1].data)

5
2 3


In [ ]:
# 두 번째 개선
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)
print(y.creator)

5


# Step 13

In [ ]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))    # z = x^2 + y^2
z.backward()
print("z = {}, x.grad = {}, y.grad = {}".format(z.data, x.grad, y.grad))

z = 13.0, x.grad = 4.0, y.grad = 6.0


# Step 14

In [ ]:
# 같은 변수 반복사용 시 오류가 발생함.
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)
y.backward()
print('x.grad', x.grad) # y = x+x = 2x 의 x에 대한 미분값은 2가 되어야 함.

y 6.0
x.grad 1.0


In [ ]:
# 오류 개선 후
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)
y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 2.0


In [ ]:
# 같은 변수를 이용하여 다른 계산시 오류가 발생
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

y = add(add(x,x),x)
y.backward()
print(x.grad)   # y = x+x+x = 3x의 미분값은 3이다.

2.0
5.0


In [ ]:
# cleargrad() 함수를 이용하여 미분값이 계속 누적되는 오류 해결
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x,x),x)
y.backward()
print(x.grad)   # y = x+x+x = 3x의 미분값은 3이다.

2.0
3.0


# Step 16

In [ ]:
# 함수를 세대 순으로 잘 꺼낼 수 있는가?
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function()  # dummy function class
    f.generation = g
    funcs.append(f)

print([f.generation for f in funcs])

funcs.sort(key=lambda x: x.generation)  # list sorting
print([f.generation for f in funcs])

f = funcs.pop()
print(f.generation)

[2, 0, 1, 4, 2]
[0, 1, 2, 2, 4]
4


In [ ]:
# 같은 함수가 중복추가되지 않는 경우
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))    # y = (x^2)^2 + (x^2)^2 = 2x^4
y.backward()

print(y.data)
print(x.grad)

[<__main__.Square object at 0x7f1f94b2c090>, <__main__.Square object at 0x7f1f94b2cf10>]
[<__main__.Square object at 0x7f1f94b2ca50>, <__main__.Square object at 0x7f1f94b2c090>]
32.0
64.0


In [ ]:
# 같은 함수가 중복 추가되는 경우 (141p 그림 16-4의 0세대 square함수가 두 번 추가된다.)
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))    # y = (x^2)^2 + (x^2)^2 = 2x^4
y.backward()

print(y.data)
print(x.grad)

[<__main__.Square object at 0x7f1f94a909d0>, <__main__.Square object at 0x7f1f94a90610>]
[<__main__.Square object at 0x7f1f94a90150>, <__main__.Square object at 0x7f1f94a909d0>]
[<__main__.Square object at 0x7f1f94a90150>, <__main__.Square object at 0x7f1f94a90150>]
32.0
128.0


# Step 17

In [ ]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=75c04b8860445ca0acc946a6eb1b43a275c728291a866e6fb73dfd199dba9112
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
from memory_profiler import memory_usage
mem_usage = memory_usage(-1, interval=1, timeout=1)
print(mem_usage)

for i in range(10):
    x = Variable(np.random.randn(10000))
    y = square(square(square(x)))
    
mem_usage = memory_usage(-1, interval=1, timeout=1)
print(mem_usage)

[175.03125]
[175.2734375]


# Step 18

In [ ]:
x0 = Variable(np.array(2.0))
x1 = Variable(np.array(2.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


In [ ]:
# 말단 변수 x0, x1 이외의 변수 y, t는 미분값이 메모리에서 삭제됨
x0 = Variable(np.array(2.0))
x1 = Variable(np.array(2.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [5]:
# 모드 전환
Config.enable_backprop = True
x = Variable(np.ones((100 ,100, 100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False
x = Variable(np.ones((100 ,100, 100)))
y = square(square(square(x)))

In [12]:
# 더 쉽게
with no_grad():
    x = Variable(np.array(2.0))
    y = square(square(square(x)))

# Step 19

In [20]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape)
print(x)

(2, 3)
variable([[1 2 3]
          [4 5 6]])


# Step 20

In [36]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a*b+c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


# Step 21

In [40]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)
